#**Assignment-1**

**Name**: Prashant Singh

**Program**: Mtech(CS)

**Enrollnment No.**: SAU/CS/Mtech(CS)/2024/04

Question: Determine whether the interval and the class target have some correlation or not? (where,
significance value = 0.1)

#**ChiMerge Function**

In [35]:
import numpy as np
import pandas as pd
from scipy.stats import chisquare
from scipy.stats import chi2_contingency

def chimerge(data, attr, target, max_intervals):
    distinct_vals = sorted(data[attr].unique())  # Unique sorted values
    targets = sorted(data[target].unique())  # Unique targets
    empty_count = {l: 0 for l in targets}  # Initialize count for targets
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))]  # Initialize intervals

    while len(intervals) > max_intervals:
        chi_values = []

        for i in range(len(intervals) - 1):
            # For Combining two intervals
            combined = data[data[attr].between(intervals[i][0], intervals[i + 1][1])]

            # Counting Occurences for each target in the combined interval
            observed_counts = [combined[target].value_counts().get(l, 0) for l in targets]
            expected_counts = [0] * len(targets)

            # Expected counts
            total = sum(observed_counts)
            if total > 0:
                for j in range(len(targets)):
                    expected_counts[j] = total * (empty_count[targets[j]] + observed_counts[j]) / total

            #Chi-squared value
            if all(count > 0 for count in expected_counts):
              chi_val = chisquare(observed_counts, f_exp=expected_counts)[0]
            else:
              chi_val = np.nan


            chi_values.append(chi_val)

        # Merging
        min_chi_index = np.argmin(chi_values)

        new_intervals = []
        skip = False
        for i in range(len(intervals)):
            if skip:
                skip = False
                continue
            if i == min_chi_index:
                new_intervals.append([min(intervals[i][0], intervals[i + 1][1]), max(intervals[i][1], intervals[i + 1][1])])
                skip = True
            else:
                new_intervals.append(intervals[i])
        intervals = new_intervals

    return intervals




#Question-1

In [36]:
#Dataset-1
data={
    'F': [1,3,7,8,9,11,23,37,39,45,46,59],
    'K':[1,2,1,1,1,2,2,1,2,1,1,1]
    }

df=pd.DataFrame(data)
print(f"The given Dataset is:\n{df}")

The given Dataset is:
     F  K
0    1  1
1    3  2
2    7  1
3    8  1
4    9  1
5   11  2
6   23  2
7   37  1
8   39  2
9   45  1
10  46  1
11  59  1


In [37]:
merged_intervals = chimerge(df, 'F', 'K', max_intervals=3)

# Creating contingency table
bins = [interval[0] for interval in merged_intervals] + [merged_intervals[-1][1]]
data['F_bins'] = pd.cut(data['F'], bins=bins, include_lowest=True)

contingency_table = pd.crosstab(data['F_bins'], data['K'])

# Chi-squared test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

if p_value < 0.1:
    print("There is a significant correlation between the interval and class label.")
else:
    print("There is no significant correlation between the interval and class label.")

print(f"Chi-squared Statistic: {chi2_stat}, P-value: {p_value}")
print(f"Contingency Table:{contingency_table}")


There is no significant correlation between the interval and class label.
Chi-squared Statistic: 2.0, P-value: 0.36787944117144245
Contingency Table:col_0          1  2
row_0              
(0.999, 39.0]  5  4
(39.0, 45.0]   1  0
(45.0, 59.0]   2  0


#Question-2

In [38]:
#Dataset-2
data2 = {
    'X': [1, 5, 9, 13, 17, 21, 24],
    'Y': [2, 8, 14, 20, 26, 32, 36],
    'C': ['A', 'B', 'A', 'B', 'A', 'B', 'A']
}

df2 = pd.DataFrame(data2)
print(f"The given Dataset is:\n{df2}")

The given Dataset is:
    X   Y  C
0   1   2  A
1   5   8  B
2   9  14  A
3  13  20  B
4  17  26  A
5  21  32  B
6  24  36  A


In [33]:
merged_intervals_X = chimerge(df2, 'X', 'C', max_intervals=3)
merged_intervals_Y = chimerge(df2, 'Y', 'C', max_intervals=3)
#Bins for X
bins_X = [interval[0] for interval in merged_intervals_X] + [merged_intervals_X[-1][1]]
bins_X = list(set(bins_X))
bins_X.sort()
#Bins for Y
bins_Y = [interval[0] for interval in merged_intervals_Y] + [merged_intervals_Y[-1][1]]
bins_Y = list(set(bins_Y))
bins_Y.sort()

data['X_bins'] = pd.cut(df2['X'], bins=bins_X, include_lowest=True)
data['Y_bins'] = pd.cut(df2['Y'], bins=bins_Y, include_lowest=True)

contingency_table_X = pd.crosstab(data['X_bins'], data['C'])
contingency_table_Y = pd.crosstab(data['Y_bins'], data['C'])
# Chi-squared test for X
chi2_stat_X, p_value_X, dof_X, expected_X = chi2_contingency(contingency_table_X)
# Chi-squared test for Y
chi2_stat_Y, p_value_Y, dof_Y, expected_Y = chi2_contingency(contingency_table_Y)

# Results for X
if p_value_X < 0.1:
    print("There is a significant correlation between X and class label.")
else:
    print("There is no significant correlation between X and class label.")

print(f"Chi-squared Statistic for X: {chi2_stat_X}, P-value for X: {p_value_X}")
print(f"Contingency Table for X:\n{contingency_table_X}")

# Results for Y
if p_value_Y < 0.1:
    print("There is a significant correlation between Y and class label.")
else:
    print("There is no significant correlation between Y and class label.")

print(f"Chi-squared Statistic for Y: {chi2_stat_Y}, P-value for Y: {p_value_Y}")
print(f"Contingency Table for Y:\n{contingency_table_Y}")


There is no significant correlation between X and class label.
Chi-squared Statistic for X: 0.0, P-value for X: 1.0
Contingency Table for X:
col_0          A  B
X                  
(0.999, 21.0]  3  3
(21.0, 24.0]   1  0
There is no significant correlation between Y and class label.
Chi-squared Statistic for Y: 0.0, P-value for Y: 1.0
Contingency Table for Y:
col_0          A  B
Y                  
(1.999, 32.0]  3  3
(32.0, 36.0]   1  0
